# Efficient ESG evaluation by NLP

チェックリストに沿ったESG評価を自然言語処理で効率化するデモです。  
チェックリストとは、ESGに関する質問のリストです。企業のレポートから適正な回答が得られた個数を数えることで評価します。質問は、Eであれば「気候変動リスクを監視する委員会があるか」「気候変動リスクと機会を特定するプロセスがあるか」などです。  
自然言語処理を用い、質問に対する回答となる箇所を自動で抽出できれば評価業務の効率化ができます。

![demo.PNG](./images/demo.PNG)




デモの手順は以下の通りです。

1. Prepare: PDFファイル(統合報告書)からテキストを読み込みます。
2. Preprocess: テキスト解析しやすいよう整形します。
3. Retrieve: 質問に関連する箇所を抽出します。単語ベースとベクトルベースの2つを行います。
4. (Optional) Answer: 関連する箇所から、質問の回答を抽出します。

### Setup for running the notebook

本Notebookを実行するには、必要なライブラリをインストールした環境が必要です。File > New > Terminalからターミナルを起動し、次のコマンドを実行してください。

* `conda env create -f env-nlp.yml`
* `conda activate env-nlp`

メニューバーのKearnel > Change Kearnel...で`env-nlp`が選択できるようになっているはずです。Kearnelは特定のNotebookを動かすための専用環境のイメージです。もし選択できない場合、次のコマンドを実行しJupyterがKernelを認識できるようにしてください。

```
ipython kernel install --user --name=env-nlp
```

`ipython`がインストールされていない場合、次のコマンドでインストールしてください。

* `conda install ipython`

作成したKernelは他のNotebookでも利用できます。本ノートブックをベースに、いろんなテキストを分析してみましょう！

## 1. Prepare

PDFファイルからテキストを読み込みます。デモでは2021年のトヨタの統合報告書を使用しています。 

In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import requests


def set_root():
    root = os.path.join(os.path.realpath("."), "../")
    if root not in sys.path:
        sys.path.append(root)
    return root

ROOT_DIR = Path(set_root())
DATA_DIR = ROOT_DIR / "data"

PDFファイルからのテキスト抽出は、事前に作成しておいた`PDFReader`クラスを使用します。内部的には[`pdfminer.six`](https://github.com/pdfminer/pdfminer.six)を使用しています。  
実装に興味がある方は[ソースコード](https://github.com/icoxfog417/esg-evaluation-by-nlp/blob/master/esg_nlp/data/pdf_reader.py)を参照してください。

In [2]:
from esg_nlp.data.pdf_reader import PDFReader
reader = PDFReader()

In [3]:
file_path = DATA_DIR / f"raw/2021_001_integrated_en.pdf"
df = reader.read_to_frame(file_path)

PDF読み込み結果を表示します。

* page: ページ番号
* order: ページ内のセクション番号(登場順にカウント)


In [4]:
df.head(5)

,page,order,content
0,0,0,Page 1
1,0,1,Integrated Report \n2021\nIntegrated Report 20...
2,1,0,Page 1
3,1,1,Contents
4,1,2,Period Covered\nFiscal 2021 (April 2020 to Mar...


## 2. Preprocess

PDF読み込み結果は様々なノイズを含んでいるので、前処理を行います。前処理として文字をすべて小文字にするなどします。

In [5]:
import re


# 小文字に統一など
preprocessed = reader.preprocess_frame(df)

`Page 1`など、タイトルのみで文を含んでいないセクションを除外します。文の判定は、ピリオドや疑問符の有無で判定します。

In [6]:
# 文を含んでいないセクションを削除
has_sentence = re.compile("(•)?\s?[A-Za-z](\s)?(\.|;)")
preprocessed = preprocessed[preprocessed["content"].apply(lambda s: re.search(has_sentence, s) is not None)]

preprocessed.head(5)

,page,order,content
3,1,2,period coveredfiscal 2021 (april 2020 to march...
6,1,5,icons that link to relevant web pages online.*...
8,1,7,the integrated report 2021 is intended to comm...
30,1,29,"on december 14, 2021, toyota held a briefing o..."
31,1,30,toyota’s part for carbon neutralityas the open...


In [7]:
print(f"Rows are decreased from {len(df)} to {len(preprocessed)}")

Rows are decreased from 1340 to 212


In [8]:
preprocessed = preprocessed.assign(length=preprocessed["content"].apply(lambda s: len(s)))
preprocessed.head(5)

,page,order,content,length
3,1,2,period coveredfiscal 2021 (april 2020 to march...,545
6,1,5,icons that link to relevant web pages online.*...,180
8,1,7,the integrated report 2021 is intended to comm...,315
30,1,29,"on december 14, 2021, toyota held a briefing o...",869
31,1,30,toyota’s part for carbon neutralityas the open...,875


## 3. Retrieve

チェックリストの質問に関係しているセクションを抽出します。単語ベースで行う手法と、ベクトルベースで行う手法を紹介します。

質問文は[CDP(Carbon Disclosue Project)](https://www.kaggle.com/c/cdp-unlocking-climate-solutions/overview)の質問を使用しました。CDPはKaggleでコンペティションを実施したことがあり、一部のデータを参照することが可能です(本当は購入しないと参照できません💦)。

In [9]:
# CDPのC2.1の質問
question = "Does your organization have a process for identifying, assessing, and responding to climate-related risks and opportunities ?"
question = question.lower()
language = "en"

人間が抽出すると、p35の箇所が該当します。

![human_annotation.PNG](./images/human_annotation.PNG)

## 3.1 Word base retrieval

単純に質問文に含まれている単語を含むセクションを抽出します。文を単語に区切るため[spaCy](https://spacy.io/)を利用します。

In [10]:
from spacy.util import get_lang_class


class Parser():

    def __init__(self, lang):
        self.lang = lang
        self.parser = get_lang_class(self.lang)()
    
    def parse(self, text):
        return self.parser(text)

/home/studio-lab-user/.conda/envs/esg-nlp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


質問から単語を抽出します。この時、ストップワードと呼ばれる一般的すぎる単語は除外します。

In [11]:
parser = Parser(language)
question_words = [str(t).strip() for t in parser.parse(question) if not t.is_stop and not re.match("\'|\.|\?|\/|\,|\-", t.text)]
question_words

['organization',
 'process',
 'identifying',
 'assessing',
 'responding',
 'climate',
 'related',
 'risks',
 'opportunities']

セクションごとにキーワードが含まれる数をカウントします。

In [12]:
def count_keyword_match(parser, keywords, text):
    tokens = parser.parse(text)
    count = 0
    for t in tokens:
        if str(t).lower().strip() in keywords:
            count += 1
    return count


counted = preprocessed.assign(
            keyword_match=preprocessed["content"].apply(
            lambda s: count_keyword_match(parser, question_words, s))
          )

In [13]:
matched = counted[counted["keyword_match"] > 0]
matched.sort_values(by=["keyword_match"], ascending=False).head(5)

,page,order,content,length,keyword_match
416,35,9,strategytoyota environmental challenge 2050toy...,2728,18
415,35,8,toyota endorsed and signed on to the recom-men...,1808,11
434,36,7,driving mode fuel efficiency) and the developm...,2033,7
226,23,9,our commitment to the above target was announc...,2801,5
492,41,10,• held company-wide events during the week of ...,3587,4


p35の文がきちんととれていることがわかります。キーワードの一致数が多い、つまり重要なセクションにフォーカスして調査をすることが可能になります。

## 3.2 Vector base retrieval

キーワードだけでなく、文章の意味を考慮したセクションの抽出を行います。文章をベクトルとして表現し、ベクトル間の距離で意味の近さを判定します。 
文章をベクトル化するために、[Googleの検索でも使用されたBERTという手法](https://blog.google/products/search/search-language-understanding-bert/)を使用します。[Hugging Face](https://huggingface.co/)というライブラリで比較的簡単に扱うことができます。

セクションは非常に長いので、文単位に分割します。

In [14]:
sentences = []
for i, row in preprocessed.iterrows():
    c = row["content"]
    for j, s in enumerate(c.replace("•", ".").replace(";", ".").split(".")):
        sentences.append({
            "page": row["page"],
            "section_order": row["order"],
            "sentence_order": j,
            "sentence": s,
            "length": len(s)
        })

sentences = pd.DataFrame(sentences)
sentences.head(5)

,page,section_order,sentence_order,sentence,length
0,1,2,0,period coveredfiscal 2021 (april 2020 to march...,52
1,1,2,1,some initiatives in fiscal 2022 (april to dec...,75
2,1,2,2,scope of reportinitiatives and activities of t...,108
3,1,2,3,", in japan and overseasreference guidelinesthi...",181
4,1,2,4,about the pdfthis file is an interactive pdf a...,103


In [15]:
len(sentences)

2301

BERTのモデルを使用し、文をベクトル表現に変換します。事前に作成しておいた`encode`関数を使用します。

In [16]:
from esg_nlp.model.encoder import encode

In [17]:
model_name = "distilbert-base-uncased"
embeddings = encode(model_name, sentences["sentence"].values.tolist())

Loading pretrained model...


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prepair the tokenizer...
Set the pipeline.
Inference start.


100%|██████████| 231/231 [02:12<00:00,  1.74it/s]


In [18]:
embeddings.shape

(2301, 768)

1717個の文それぞれに対し、768次元のベクトルが得られました。質問文もベクトルにします。

In [19]:
query = encode(model_name, question)
query = np.reshape(query, (1, -1))

Loading pretrained model...


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prepair the tokenizer...
Set the pipeline.
Inference start.


In [20]:
query.shape

(1, 768)

質問文のベクトルと、各文のベクトルとの距離を算出します。

In [21]:
from sklearn.metrics.pairwise import cosine_similarity


distance = cosine_similarity(query, embeddings)
measured = sentences.assign(
            distance=distance.flatten())

measured.sort_values("distance", ascending=False, inplace=True)
measured.head(10)

,page,section_order,sentence_order,sentence,length,distance
1831,43,7,0,through this process of bcp formulation and re...,128,0.913500
1196,31,8,1,building good relationships with all stakehol...,246,0.886414
1627,39,10,1,working together with suppliers on risk monit...,181,0.885914
873,22,8,11,"to realize carbon neutrality, we need to expa...",171,0.883928
1416,35,9,2,"among these risks and opportunities, climate ...",176,0.876290
1471,36,7,8,risk management relating to climate changewe s...,254,0.874763
1420,35,9,6,"in accordance with this understanding, we hav...",236,0.872937
1628,39,10,2,"furthermore, we work with ngos and other exte...",173,0.868915
1411,35,8,5,"moreover, the sustainability meeting, which ...",346,0.868187
1467,36,7,4,to confirm the validity and progress of toyot...,231,0.863832


p35以外の、あまり関係ない箇所の文書もリストアップされています。厳密に文を取りたい場合にはノイズが多くなりそうです。その意味では、ベクトル検索で取得してからキーワードで絞り込んだ方が有効に見えます。

## 4.(Optional) Answer:

質問回答を学習させたモデルを利用して、直接的に回答を抽出する手法もあります。キーワードで絞り込んだデータから回答を抽出してみましょう。

回答箇所の抽出には、自然言語処理の質問回答の手法を使用します。Wikipediaをベースにした質問回答のデータセット([SQuAD](https://rajpurkar.github.io/SQuAD-explorer/)と呼ばれる)で事前に学習したモデルをお持ちいます。本来はESGに関する質問と回答のデータセットで転移学習したほうが良いですが、今回は学習せずに用います。

In [22]:
from esg_nlp.model.question_answer import answer

1つはキーワードにマッチしているセクションを対象にします。

In [23]:
minimum_match = 1
question_section_pair = matched[matched["keyword_match"] >= minimum_match]["content"].apply(lambda s: (question, s)).tolist()
len(question_section_pair)

61

In [24]:
answers = answer("distilbert-base-uncased-distilled-squad", question_section_pair)
pd.DataFrame(answers).sort_values(by="score", ascending=False).head(5)

Loading pretrained model...
Prepair the tokenizer...
Set the pipeline.
Answer start.


  2%|▏         | 1/61 [00:00<00:10,  5.87it/s]/home/studio-lab-user/.conda/envs/esg-nlp/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:705: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/studio-lab-user/.conda/envs/esg-nlp/lib/python3.8/site-packages/transformers/pipelines/question_answering.py:295: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  p_mask = np.asarray(
100%|██████████| 61/61 [00:15<00:00,  3.91it/s]


,score,start,end,answer
19,0.597159,2044,2076,we will conduct the verification
29,0.380538,611,620,increased
15,0.362809,1742,1802,does not indicate the performance of vehicles ...
37,0.235122,617,681,as one of the first companies to respond to cl...
50,0.229182,138,193,evaluating ease of getting in and out with a w...


`score`が高い回答があるものの、気候変動の機会・リスクに関する回答とはなっていません。ESGに関する質問は一言で回答できるものではないので、そのまま使うのは難しいかもしれません。

デモは以上になります。統合報告書のPDFや質問の文書を変えて試してみてください！